In [ ]:
# Installing Necessary Packages
!pip install PyMySQL
!pip install mysql-connector-python
!pip install sqlalchemy
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine
import datetime
import pymongo
import pprint
import json
from bson import json_util

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Specifying/Loading My SQL ID's
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "project1"

In [ ]:
# Defining Authentication Details
atlas_cluster_name = "cluster0.0zywi.mongodb.net"
atlas_default_dbname = "test"
atlas_user_name = "ds2002"
atlas_password = "UVA!1819"

In [ ]:
# Connecting to MySQL and creating a cursor object for queries
try: 
    conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name) 
    cursor = conn.cursor()

    df = pd.read_sql("SELECT * FROM table_name;", conn)

    conn.close()
    df.head()
except Exception as e: 
    print(f"Error connecting to host")

In [ ]:
# Establishing MongoDB
try: 
  conn_str = {
  #  "local" : f"mongodb://{host_name}:{port}/",
    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}/{atlas_default_dbname}" 
}

  client = pymongo.MongoClient(conn_str["atlas"])

  #print(f"Local Connection String: {conn_str['local']}")
  print(f"Atlas Connection String: {conn_str['atlas']}")
except Exception as e: 
  print(f"Error forming Atlas String Connection: {conn_str['atlas']}")

In [ ]:
# Accessing MongoDB and retrieving sample_weatherdata
try: 
    client.list_database_names()
    db = client['sample_weatherdata']
    wd = client.sample_weatherdata
    weatherdata = wd.data
except Exception as e: 
    print(f"Error accessing database")

In [ ]:
# Connect to SQL database and query data
try: 
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sql_engine = create_engine(conn_str)
    df_sql = pd.read_sql("SELECT * FROM table_name", con=sql_engine)
except Exception as e:
    print(f"Error accessing SQL database")

# Connect to MongoDB database and query data
try: 
    mongo_client = pymongo.MongoClient(f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}/{atlas_default_dbname}")
    mongo_db = mongo_client["sample_weatherdata"]
    mongo_collection = mongo_db["data"]
    mongo_data = list(mongo_collection.find())
    df_mongo = pd.DataFrame(mongo_data)

    df_mongo = df_mongo.applymap(lambda x: str(x) if not isinstance(x, str) else x)
except Exception as e: 
    print(f"Error accessing MongoDB or Querying Data")

# Write dimension table to SQL database
try: 
    df_mongo.to_sql("dim_weather", con=sql_engine, if_exists="replace")
    df_mongo.head()
except Exception as e: 
    print(f"Error writing dimension table to SQL database")

In [ ]:
# This file creates a dimension table using 'sample_weather data' as loaded from MongoDB. 
# This specific dataset was used to see if certain weather conditions affect flight departures/cause delays and verify if there is a pattern of certain weather conditions and delays (example: hurricane season could increase delays).